In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import random
random.seed(420)


url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_test.data'
df_test = pd.read_csv(url, header=None, sep =' ')
df_test = df_test.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.data'
X_train = pd.read_csv(url, header=None, sep =' ')
X_train = X_train.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.labels'
y_train = pd.read_csv(url, header=None)

from sklearn.preprocessing import LabelEncoder

y_train2 = LabelEncoder().fit_transform(y_train)
y_train2

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier(random_state=21)

In [ ]:
from scipy.stats import uniform, randint


grid_params_rs = [{
    "n_estimators": randint(1,5000),
    "learning_rate": uniform(0,10),
    "max_depth": randint(1,15),
    "min_child_weight": uniform(0,7)
}]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

results_rs = pd.DataFrame()
grid = RandomizedSearchCV(clf,grid_params_rs, scoring='balanced_accuracy', random_state=37, n_iter=50)

In [ ]:
grid.fit(X_train, y_train2)

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None...
                   param_distributions=[{'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa4d1f68640>,
                                         'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa4d1f69c30>,
                                         'min_child_weight': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa4d1f69360>,
                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa4d1f37c10>}],
                   random_state=37, scoring='roc_auc')

In [ ]:
results_rs["params"] = grid.cv_results_["params"]
results_rs["a_scores"] = grid.cv_results_["mean_test_score"]

In [ ]:
results_rs.to_csv("xgb_results_rs.csv",index=False)